#Silver Layer Scripting : Transformation Notebook


This notebook focuses exclusively on transforming the **product's information** dataset from the Bronze layer into a clean and trusted Silver table.
Each transformation ensures data quality, consistency, and analytics readiness

**Dataset full Name** : bike_lakehouse.bronze.crm_prd_info


###Load functions and libraries

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.functions import col , trim 
from pyspark.sql.types   import StringType

###Load Bronze Table 
Read the Bronze table into a Spark DataFrame to begin transformations.

In [0]:
df = spark.table("bike_lakehouse.bronze.crm_prd_info")

df.limit(10).display()

prd_id,prd_key,prd_nm,prd_cost,prd_line,prd_start_dt,prd_end_dt
210,CO-RF-FR-R92B-58,HL Road Frame - Black- 58,null,R,2003-07-01,null
211,CO-RF-FR-R92R-58,HL Road Frame - Red- 58,null,R,2003-07-01,null
212,AC-HE-HL-U509-R,Sport-100 Helmet- Red,12,S,2011-07-01,2007-12-28
213,AC-HE-HL-U509-R,Sport-100 Helmet- Red,14,S,2012-07-01,2008-12-27
214,AC-HE-HL-U509-R,Sport-100 Helmet- Red,13,S,2013-07-01,null
215,AC-HE-HL-U509,Sport-100 Helmet- Black,12,S,2011-07-01,2007-12-28
216,AC-HE-HL-U509,Sport-100 Helmet- Black,14,S,2012-07-01,2008-12-27
217,AC-HE-HL-U509,Sport-100 Helmet- Black,13,S,2013-07-01,null
218,CL-SO-SO-B909-M,Mountain Bike Socks- M,3,M,2011-07-01,2007-12-28
219,CL-SO-SO-B909-L,Mountain Bike Socks- L,3,M,2011-07-01,2007-12-28


###Trim String Columns
Automatically remove leading/trailing spaces from all string columns.

In [0]:
for field in df.schema.fields :
    if isinstance(field.dataType,StringType) :
       df = df.withColumn(field.name , trim(col(field.name)))

###Normalize Categorical Columns
Convert coded values to readable, standardized categories.


In [0]:
df = (

    df.withColumn(
        "prd_line" ,
        F
        .when(F.upper(col("prd_line")) == 'S' , 'Other Sales')
        .when(F.upper(col("prd_line")) == 'M' , 'Mountain'   )
        .when(F.upper(col('prd_line')) == 'T' , 'Touring'    )
        .when(F.upper(col('prd_line')) == 'R' , 'Road'       )
        .otherwise('n/a')
    )
)

###Remove Rows with Null Keys
Filter out records with null primary keys to maintain referential integrity.

In [0]:
df = df.filter(col('prd_id').isNotNull())

###Product Key Parsing
Extracts category ID from composite key and reformat the prd_id

In [0]:
# Extracts category ID : cat_id from the product Key : prd_key using regex
df = df.withColumn("cat_id" , F.regexp_replace(F.substring(col("prd_key"),1,5),'-','_'))

# Extract Clean Product Key : Starts at position 7 and extracts the rest.
df = df.withColumn("prd_key", F.substring(col("prd_key"),7,F.length(col('prd_key'))))

###Cost Cleanup  Handling Null Values
Uses coalesce() to replace nulls and Ensures numeric stability

In [0]:
df = df.withColumn("prd_cost" , F.coalesce(col("prd_cost"),F.lit(0)))

### Date Casting Transformation
The following line converts the column prd_start_dt into a proper Spark Date data type.

In [0]:
from pyspark.sql.types import DateType

df = df.withColumn('prd_start_dt' , col('prd_start_dt').cast(DateType()))

###Rename Columns
Standardize column names across the dataset using a mapping dictionary.

In [0]:
RENAME_MAP = {
    "prd_id": "product_id",
    "cat_id": "category_id",
    "prd_key": "product_number",
    "prd_nm": "product_name",
    "prd_cost": "product_cost",
    "prd_line": "product_line",
    "prd_start_dt": "start_date",
    "prd_end_dt": "end_date"
}

In [0]:
for old_name , new_name in RENAME_MAP.items() :
    df = df.withColumnRenamed(old_name , new_name)

###Sanity checks of dataframe
Quickly check the result of transformations, before moving forward with the dataFrame

In [0]:
df.limit(10).display()

product_id,product_number,product_name,product_cost,product_line,start_date,end_date,category_id
210,FR-R92B-58,HL Road Frame - Black- 58,0,Road,2003-07-01,null,CO_RF
211,FR-R92R-58,HL Road Frame - Red- 58,0,Road,2003-07-01,null,CO_RF
212,HL-U509-R,Sport-100 Helmet- Red,12,Other Sales,2011-07-01,2007-12-28,AC_HE
213,HL-U509-R,Sport-100 Helmet- Red,14,Other Sales,2012-07-01,2008-12-27,AC_HE
214,HL-U509-R,Sport-100 Helmet- Red,13,Other Sales,2013-07-01,null,AC_HE
215,HL-U509,Sport-100 Helmet- Black,12,Other Sales,2011-07-01,2007-12-28,AC_HE
216,HL-U509,Sport-100 Helmet- Black,14,Other Sales,2012-07-01,2008-12-27,AC_HE
217,HL-U509,Sport-100 Helmet- Black,13,Other Sales,2013-07-01,null,AC_HE
218,SO-B909-M,Mountain Bike Socks- M,3,Mountain,2011-07-01,2007-12-28,CL_SO
219,SO-B909-L,Mountain Bike Socks- L,3,Mountain,2011-07-01,2007-12-28,CL_SO


### Write Silver Table
Persist the cleaned DataFrame as a Delta table in the Silver layer.

In [0]:
df.write.mode("overwrite").format("delta").saveAsTable("bike_lakehouse.silver.crm_products")

In [0]:
%sql
select * from bike_lakehouse.silver.crm_products limit 10 ;

product_id,product_number,product_name,product_cost,product_line,start_date,end_date,category_id
210,FR-R92B-58,HL Road Frame - Black- 58,0,Road,2003-07-01,null,CO_RF
211,FR-R92R-58,HL Road Frame - Red- 58,0,Road,2003-07-01,null,CO_RF
212,HL-U509-R,Sport-100 Helmet- Red,12,Other Sales,2011-07-01,2007-12-28,AC_HE
213,HL-U509-R,Sport-100 Helmet- Red,14,Other Sales,2012-07-01,2008-12-27,AC_HE
214,HL-U509-R,Sport-100 Helmet- Red,13,Other Sales,2013-07-01,null,AC_HE
215,HL-U509,Sport-100 Helmet- Black,12,Other Sales,2011-07-01,2007-12-28,AC_HE
216,HL-U509,Sport-100 Helmet- Black,14,Other Sales,2012-07-01,2008-12-27,AC_HE
217,HL-U509,Sport-100 Helmet- Black,13,Other Sales,2013-07-01,null,AC_HE
218,SO-B909-M,Mountain Bike Socks- M,3,Mountain,2011-07-01,2007-12-28,CL_SO
219,SO-B909-L,Mountain Bike Socks- L,3,Mountain,2011-07-01,2007-12-28,CL_SO
